In [3]:
import sys
sys.path.insert(0, "../../")


In [4]:
import os 
import frontmatter
from dataclasses import dataclass, field

def read_frontmater(file_path):
    # Create temp file

    with open(test_file_path) as f:
        post = frontmatter.load(f)

    fm = {key: post.get(key) for key in post.keys()}
    fm.update({'urls': []})

    for kb in fm.get('linked_kbs'):
        # url = re.search("(?P<url>https?://[^\s]+)", kb).group("url")
        t = kb.split('](')
        t = t[0][1:]
        fm['urls'].append(t)

    del fm['linked_kbs']

    return fm


test_file_path = "kb_md/posts/10000ft_connector/index.qmd"

class PathExists(Exception):
    def __init__(self):
        super().__init__("path does not exist")


if not os.path.exists(test_file_path):
    raise  PathExists(Exception)
    
test_fm = read_frontmater(test_file_path)
test_fm.keys()

TypeError: __init__() takes 1 positional argument but 2 were given

In [ ]:
import datacrew.crawler.crawler as dcc
driver = dcc.driversetup(is_headless=False)


In [ ]:
from selenium.webdriver.common.by import By

TOPIC_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US"
TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US'

BASE_URL = "https://domo-support.domo.com/s/topic/"

test_page_source = dcc.pagesource(url=TOPIC_URL, driver=driver,
                                  element_type=By.CLASS_NAME, element_id="section-list-item"
                                  )

# test_page_source


💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US is loaded within 10 seconds.


In [ ]:
import datacrew.crawler.article as dca

BASE_TOPIC_URL = "https://domo-support.domo.com/s/topic/"
TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US'

pprint(dca.Article_Category(url=TOPIC_URL,
                 driver=driver,
                 base_url=BASE_TOPIC_URL))

💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US is loaded within 10 seconds.
Article_Category(base_url='https://domo-support.domo.com/s/topic/',
                 linked_url_ls=['https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaoQGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZapAGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaoVGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaojGAC',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaoyGAC',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZapVGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w

In [ ]:
import os

kb_path = "nbs/projects/kb_md/posts"
kb_path = "kb_md/posts/"

if not os.path.exists(kb_path):
    raise Exception('kb_path does not exist')

for path, subdirs, files in os.walk(kb_path):
    for name in files:
        if name != 'index.qmd':
            continue
        # print(path, name)
        file_path = os.path.join(path, name)
